In [1]:
import os
# Azure OpenAI keys
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://adhoc-south-central-us.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "f9f14bee8b8b47b0a25ed553832017ef"
os.environ["SERPAPI_API_KEY"] = "dc8560acaac2ef0a43805d078fff52b94480571b43fe894344318692e963570c"
DEPLOYMENT = "gpt-4"
MODEL = "gpt-4"
TEMPERATURE = 0

In [2]:
# import text loader to 
from langchain.document_loaders.word_document import Docx2txtLoader

from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

C:\Users\sonishubham\Anaconda3\envs\auto-gpt-2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sonishubham\Anaconda3\envs\auto-gpt-2\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
C:\Users\sonishubham\Anaconda3\envs\auto-gpt-2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
## Get all filepaths inside BuildKeyNotes Folder
import os

def get_filepaths(folder_path):
    file_paths = []
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

# Provide the folder path
folder_path = "BuildKeyNotes"

# Call the function to get the file paths
filepaths = get_filepaths(folder_path)
# Print the file paths
for filepath in filepaths:
    print(filepath)

BuildKeyNotes\BRK214H.docx
BuildKeyNotes\BRK223H.docx
BuildKeyNotes\BRKFP303.docx
BuildKeyNotes\KEY01H.docx
BuildKeyNotes\KEY02H.docx


In [4]:
loader = Docx2txtLoader(file_path="BuildKeyNotes\BRK214H.docx")

In [5]:
docs = loader.load_and_split()

In [6]:
docs

[Document(page_content="Omkar\n\nS.no\n\nQuestion\n\nTprompt\n\nTitle + NoAgent\n\nTitle Index  + Agent\n\nSummary Index + Agent\n\n1\n\nHow do I stop billing on my azureml Compute Instance?\n\nIncorrect\n\n3\n\n1\n\n2\n\n2\n\nHow do I deploy an AzureML Model?\n\nIncorrect\n\n3\n\n2\n\n1\n\n3\n\nHow can I get started with Azure Machine Learning Studio? \n\n1\n\n4\n\n3\n\n2\n\n4\n\nHow do I deploy mlflow models to an online endpoint?\n\n1\n\n4\n\n3\n\n2\n\n5\n\nWhat is Automated Machine Learning?\n\n1\n\n1\n\n1\n\n1\n\n6\n\nHow do I install AzureML SDK V2?\n\nIncorrect\n\nIncorrect\n\nIncorrect\n\n1\n\n7\n\nWhat is Compute Instance?\n\n1\n\n1\n\n1\n\n2\n\n8\n\nHow to create a command job in AzureML?\n\nIncorrect\n\n2\n\n1\n\nIncorrect\n\n9\n\nHow to attach a kubernetes cluster to a workspace?\n\n2\n\n2\n\n2\n\n2\n\n10\n\nHow can I add a Managed Identity to my compute cluster?\n\n\xa0\n\n\xa0\n\n\xa0\n\n\xa0\n\n11\n\nWhen should I use responsible AI\n\n2\n\n2\n\n1\n\n1\n\nMicrosoft Build

In [143]:
# Now we can override it and set it to "AI Assistant"
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import AzureChatOpenAI
chat = AzureChatOpenAI(
    deployment_name="gpt-4",
    model_name=MODEL,
    temperature=TEMPERATURE
)
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI Assistant:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=template
)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=chat, 
    verbose=False, 
    memory=ConversationBufferMemory(ai_prefix="AI Assistant")
    
)

In [144]:
conversation.predict(input ="hi",)

"Hello! How can I help you today? Is there something specific you'd like to talk about or ask?"

In [146]:
print(colored("AI Response:", tr(conversation.memory.chat_memory.messages),"green"))

NameError: name 'colored' is not defined

In [135]:
PROMPT

PromptTemplate(input_variables=['history', 'input'], output_parser=None, partial_variables={}, template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI Assistant:', template_format='f-string', validate_template=True)

In [7]:
from langchain.embeddings import OpenAIEmbeddings
# Define your embedding model
embeddings_model = OpenAIEmbeddings(
    deployment="text-embedding-ada-002",
    model="text-embedding-ada-002",
    chunk_size=1
)

In [8]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings_model,
)

In [9]:
query = "customers that are using Azure OpenAI today"
docs = db.similarity_search(query)

In [10]:
docs

[Document(page_content="Microsoft Build: Getting started with generative AI using Azure OpenAI Service\n\nDom Divakaruni, Pablo Castro, Jamie Ovenden, Charlotte Wood, Tim Gordon\n\nMay 2023\n\n\n\n[MUSIC] \n\n\n\nDOM DIVAKARUNI: Well, hello everyone. Great to see you. My name is Dom Divakaruni, I lead Product for Azure OpenAI, and I'm joined by my esteemed colleague, Pablo Castro, who leads Azure Cognitive Search. I'm super happy to be here with you all here to talk about Azure OpenAI and the tools that we're building to help you develop applications with it, to develop Copilots with it, and excited to share a bunch of new announcements, some of which you've heard during Scott's talk, and dive into it in some more details here.\n\n\n\nYou've heard a lot about this, but again, setting the stage, the field is as old as some of us here in the room and it's really been accelerating since 2012, started with the AlexNet Paper and then the transformers in 2017 but really it's picked up steam 

In [11]:
retriever = db.as_retriever()

In [18]:
beyonce = """Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."""

In [29]:
satya = """Another area that I want to share is plugins. We heard about plugins before. What are we doing in Azure OpenAI to help customers leverage plugins? Some of the problems we wanted to tackle with these models is where these models fall short today. They're great with lots and lots of different languages. Probably fall short a little bit in some tier two, tier three languages, and some Asian languages, African languages, etc. OpenAI also has something called the retrieval plugin that talks to multiple vector databases. We wanted to also incorporate that as a first-class thing with Azure OpenAI.\n\n\n\nLastly, we wanted to tie Azure OpenAI to information on the web so you can ground it with more relevant recent information as that's what's most useful, and build applications on top of all of this. And so happy to introduce Azure OpenAI Service Plugins, where this will be interoperable with all the plugins that OpenAI has, but what we're focused on first is to solve some of these key problems by integrating these Azure services or Microsoft services into our plugins to allow for these models to be extended for you to build applications on these models, extend them out to these various Azure services. All these sources are managed with Azure AD and managed identities, so your application can only access stuff that it's authorized to. Then additionally, because of some of the newness and the potential and learnings in the space, we want to ensure that we provide administrators with the control to only enable plugins that are approved in your enterprise for your Azure OpenAI resources.\n\n\n\nYou can administer what folks can see and what folks can't see. I'm going to quickly demo this and hopefully, the demo gods smile on me this time. At first, you don't succeed. In Azure OpenAI, I have a new section for plugins where I can enable the stuff. As you can see, I have three plugins enabled: Bing Search, Translator, and Cognitive Search, what Pablo was just demoing, and so I'm going to pretend here that we've built a Copilot that faces Contoso manufacturing corporation, and there's a problem with a part. There's a customer complaint that I'm trying to figure out what the customer is saying and help them resolve it. In this scenario, a customer e-mail came in in Vietnamese, and I don't know if I want to trust the model quality in trying to parse out the customer's concerns, so I want to use Azure Translator, which has really high translation accuracy. I go ahead and send that in and I want to make sure that it not only translates, but the model then acts on the translation to help summarize what are the client's most key concerns. We wait for this to come in."""

In [14]:
random = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla non semper urna, eget convallis mauris. Fusce id massa condimentum, lobortis justo et, tincidunt diam. Aenean malesuada velit at felis malesuada, vitae cursus sem ultricies. Quisque ac eleifend nisi, nec lacinia nulla. Duis iaculis nisl eu aliquet sollicitudin. Vivamus efficitur libero quis purus iaculis, at cursus est viverra. Nulla aliquet tortor in tortor pulvinar, in feugiat ipsum commodo. Cras interdum enim nec felis congue malesuada. Nulla facilisi. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; In vitae nunc ultrices, lobortis quam in, auctor libero. Nulla facilisi. Mauris sit amet varius nunc. Donec sit amet lectus vitae sapien luctus blandit. Aenean semper est ac quam aliquam, id lobortis dui hendrerit"""

In [45]:
dom = """My name is Dom Divakaruni, I lead Product for Azure OpenAI, and I'm joined by my esteemed colleague, Pablo Castro, who leads Azure Cognitive Search. I'm super happy to be here with you all here to talk about Azure OpenAI and the tools that we're building to help you develop applications with it, to develop Copilots with it, and excited to share a bunch of new announcements, some of which you've heard during Scott's talk, and dive into it in some more details here.\n\n\n\nYou've heard a lot about this, but again, setting the stage, the field is as old as some of us here in the room and it's really been accelerating since 2012, started with the AlexNet Paper and then the transformers in 2017 but really it's picked up steam in the last couple of years. None of us have seen this pace of innovation and it's really changing all our lives in such a meaningful way very quickly. Seen in no other than ChatGPT in how quickly it really captured all of our attention. In no time in history has technology or really any innovation captured all our interests so quickly and has reached the adoption levels of ChatGPT as we've seen. In really two months, it has scaled to 100 million users, and it just speaks to how useful and practical it is in all our daily lives and intuitive.\n\n\n\nWe at Azure AI are happy to build all these innovations into products to make them second nature for us to consume in our productivity suites, in search engines that we use as well as low-code, no-code tools that we all use in PowerBI and PowerApps. All that can zoom the powerful AI models in the customizable AI model, including Azure OpenAI, which is our service that brings together OpenAI's most recent innovation to life. You can build applications with it because that's what's most important. All of this runs on Azure Machine Learning, our massive-scale AI platform that helps all these innovations come to life.\n\n\n\nWe look at Azure OpenAI. What is it? It is our platform that helps us bring OpenAI's research to life. It rides on a foundation of enterprise security with all the highest security standards and what you expect from the cloud and what you can depend on, where your data is private and it remains in your tenant, and none of it is used to train or improve our systems or train machine learning models, and provides you with all of OpenAI's models as quickly as we possibly can, including the most recent GPT-4 model. It has really captured all the attention of the industry. It's lighting up all these unique use cases that haven't been possible before: summarization, reasoning over data, writing tools, co-generation, and now ushering in the era of Copilots.\n\n\n\nGPT-4 is the latest to the state of the art from OpenAI and you've heard a lot about it. It achieves human-level perf on text generation. It lets you steer it with nuanced instructions. Actually, OpenAI will tell you that they've had this model done training and still in development for about six months. That time was carefully spent tuning the model to make it more steerable and useful when you instruct it to do something. It really shows.I don't know if you all have had a chance to play with GPT4 either in ChatGPT or the Azure OpenAI Service, but what you're able to do with it is really remarkable. There was a time in the past where prompt engineering really was considered an art and it really took a lot of coaxing to get the model to do what you need it to do. Not anymore with GPT4. Just with a few instructions, what you can create with it is fricking phenomenal. All of that is really being used by thousands of customers. I think Scott cited 4,500 customers that are using Azure OpenAI today and really building with it. Coursera is a great example of a customer that is building AI-powered learning experience on our platform to help learners with an interactive learning experience that helps learning be a more personalized journey than one that is generic and crafted, and that's out there and we have to adapt to it.\n\n\n\nAs we look at this, we see three use cases really emerging. One is where AI is embedded into products we use like search and makes things fundamentally better. Search synthesis as opposed to search results on a page that you have to discern what the right thing is for you, and pretty simple use cases like generating blogs or helping us write emails. The next level above that is what we've seen just over the past few months with ChatGPT, where it's helping novices learn topics that they are not familiar with with a simple QA-based style learning. That really opens the doors for us to become more effective at everything we do by just simply asking the model to help be our Copilot to guide us along a task.\n\n\n\nThe last part where we're seeing these models really help and emerges to help experts be more efficient at what they do, offloading their tasks and helping the knowledge worker be more effective at what they do. We're seeing that in no other place. Epic Healthcare plays a major role in the healthcare industry, and we're helping them incorporate GPT-4 into electronic health records. They are helping medical professionals use their time more effectively. Another great example of helping the expert is DataRobot. DataRobot, our partner, incorporated Azure OpenAI to help their customer cohort, their customer base of data scientists and ML practitioners be more effective"""

In [97]:
poem ="""In the depths of my solitude, I wander,
A lone traveler in the realm of dreams.
Whispers of the night caress my weary soul,
As I dance amidst the moonbeams.

The stars above, they twinkle and sigh,
A celestial ballet upon the velvet sky.
Their shimmering light, a guiding force,
Leading me to secrets, a hidden source.

I weave through the tapestry of time,
Where ancient echoes and memories chime.
Visions of love, like blossoms in spring,
Flutter softly on the wings of a zephyr's wing.

Yet, amidst the beauty, a tempest brews,
A storm of passion, a fiery fuse.
In the depths of my heart, a fire burns,
Yearning for love's embrace, as the world turns.

Through the winding paths of fate, I tread,
With bated breath and hopes widespread.
In this symphony of life, a dance unfolds,
As melodies of love and longing, forever enfold.

Oh, let me be the poet of the night,
Dancing with stars, embracing their light.
Through solitude's embrace, my spirit shall rise,
In a dance of dreams, where two poems intertwine."""

In [15]:
questions = """
questions_answers": [

    {
      "question": "What did Beyoncé do as a child?",
      "answer": "She performed in various singing and dancing competitions"
    }
  ]"""

In [98]:
PLANNING_SYSTEM_CONTEXT = """

Given the following text:

"{text}"

Please evaluate the text based on the following factors and generate a confidence score between 0 and 1 to rate its suitability for generating high-quality question-answer pairs. Consider the following guidelines for each factor:

1. Clarity and Coherence: Assess the overall clarity and coherence of the text. Is the information presented in a clear and concise manner? Are the ideas logically structured and easy to follow?

2. Completeness of Information: Evaluate whether the text provides comprehensive information on the topic. Does it cover the subject matter extensively and include relevant details?

3. Unambiguous Language: Determine if the text uses unambiguous language and avoids excessive jargon or technical terms. Is the terminology consistent and understandable?

4. Depth and Complexity: Consider the complexity of the text and the depth of the information provided. Does it offer deeper insights, nuanced explanations, or detailed analyses?

5. Presence of Factual Information: Check for the presence of verifiable and factual information within the text. Does it provide accurate and reliable information?

6. Contextual Understanding: Evaluate whether the text provides sufficient contextual information that can be used to generate relevant and meaningful questions. Does it include examples, comparisons, or explanations to aid understanding?

7. Consistency and Accuracy: Assess the consistency and accuracy of the information presented in the text. Are there any inconsistencies, contradictions, or factual errors that could affect the quality of question-answer pairs?

8. Structure and Formatting: Consider the organization and formatting of the text. Does it have clear headings, subheadings, or bullet points that help structure the content?

Based on your evaluation of these factors, provide an overall confidence score between 0 and 1, where 0 indicates low confidence, 0.5 average confidence and 1 indicates high confidence.
Rate lower when you cannot generate question answers pairs from it.

Confidence Score: 


"""

In [72]:
PLANNING_SYSTEM_CONTEXT = """
You are an AI designed to help generate dataset for foundational LLMs.

You can help generating question answering dataset for foundational models by analyzing the text
and recommend different types of questions to ask about the
text.

You should analyze the text content that is delimited by triple backticks.

Your first task is to analyze the text and break it down into logical angles to
ask questions about the text. The angles chosen should be chosen based on the
type of text provided.For example, If the text is a news article, the angles should be
different than if the text is a research paper.

Here are some examples of angles you could take but are not limited to:
1. Surface level questions such as subjects and actions
2. Deeper questions such as the implications of the text and the future of the text
3. Emotions of the text and speakers
4. The context of the text and the speakers
5. Synthesizing multiple parts of the text
6. and more!

Your response should be a list of different types of questions to ask about the
text. The items in the list should cover various parts of the text and should
not be redundant. Together, the questions created by the items in the list
should be as exahuastive as possible. The questions in the list should be
somewhat specific to the text at hand

You should only respond using the following format below. Ensure the response can be parsed by Python json.loads:
[
    "question type 1",
    "question type 2",
    "question type 3",
    ...
]

Your end goal is to generate generic questions types that can cover whole text, which will be used to generate more specific questions about the given text.

text: ```{text}```

"""

In [84]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(PLANNING_SYSTEM_CONTEXT)


In [85]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='\n\nGiven the following text:\n\n"{text}"\n\nPlease evaluate the text based on the following factors and generate a confidence score between 0 and 1 to rate its suitability for generating high-quality question-answer pairs. Consider the following guidelines for each factor:\n\n1. Clarity and Coherence: Assess the overall clarity and coherence of the text. Is the information presented in a clear and concise manner? Are the ideas logically structured and easy to follow?\n\n2. Completeness of Information: Evaluate whether the text provides comprehensive information on the topic. Does it cover the subject matter extensively and include relevant details?\n\n3. Unambiguous Language: Determine if the text uses unambiguous language and avoids excessive jargon or technical terms. Is the terminology consistent and understandable?\n\n4. Depth and Complexity: Consider the complexity of the text and the dep

In [86]:
prompt_template.messages[0].prompt.input_variables

['text']

In [115]:
customer_messages = prompt_template.format_messages(text= beyonce, questions_answers = questions
                   )

In [116]:
customer_messages

[HumanMessage(content='\n\nGiven the following text:\n\n"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy"."\n\nPlease evaluate the text based on the following factors and generate a confidence score between 0 and 1 to rate its suitability for generating high-quality question-answer pairs. Consider the following guidelines for each factor:\n\n1. Clari

In [117]:
# Call the LLM to translate to the style of the customer message
from langchain.chat_models import AzureChatOpenAI
chat = AzureChatOpenAI(
    deployment_name="gpt-4",
    model_name=MODEL,
    temperature=TEMPERATURE
)
customer_response = chat(customer_messages)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists..


In [118]:
questionTypes = customer_response.content

In [119]:
questionTypes

'Confidence Score: 0.85'

In [104]:
################ HUMMAN FEEDBACK LOOP HERE #####################################  TODO

In [65]:
# Get relevant docs and summarize it for a chunk

In [186]:
relevant_docs = db.similarity_search(docs[1].page_content)

In [187]:
relevant_docs

[Document(page_content="I don't know if you all have had a chance to play with GPT4 either in ChatGPT or the Azure OpenAI Service, but what you're able to do with it is really remarkable. There was a time in the past where prompt engineering really was considered an art and it really took a lot of coaxing to get the model to do what you need it to do. Not anymore with GPT4. Just with a few instructions, what you can create with it is fricking phenomenal. All of that is really being used by thousands of customers. I think Scott cited 4,500 customers that are using Azure OpenAI today and really building with it. Coursera is a great example of a customer that is building AI-powered learning experience on our platform to help learners with an interactive learning experience that helps learning be a more personalized journey than one that is generic and crafted, and that's out there and we have to adapt to it.\n\n\n\nAs we look at this, we see three use cases really emerging. One is where A

In [188]:
# combined relevant docs
qdocs = "".join([docs[i].page_content for i in range(len(relevant_docs))])

In [189]:
qdocs

"Microsoft Build: Getting started with generative AI using Azure OpenAI Service\n\nDom Divakaruni, Pablo Castro, Jamie Ovenden, Charlotte Wood, Tim Gordon\n\nMay 2023\n\n\n\n[MUSIC] \n\n\n\nDOM DIVAKARUNI: Well, hello everyone. Great to see you. My name is Dom Divakaruni, I lead Product for Azure OpenAI, and I'm joined by my esteemed colleague, Pablo Castro, who leads Azure Cognitive Search. I'm super happy to be here with you all here to talk about Azure OpenAI and the tools that we're building to help you develop applications with it, to develop Copilots with it, and excited to share a bunch of new announcements, some of which you've heard during Scott's talk, and dive into it in some more details here.\n\n\n\nYou've heard a lot about this, but again, setting the stage, the field is as old as some of us here in the room and it's really been accelerating since 2012, started with the AlexNet Paper and then the transformers in 2017 but really it's picked up steam in the last couple of y

In [190]:
SUMMARIZE_SYSTEM_CONTEXT = """
You are an AI designed to summarize text.

Your job is provide a summary of the text text content that is delimited by triple backticks.
Summary should be pointed to specific points, shouldn't be creative and cover as much as possible the text.

It Should contains jargons and creative thoughts.
Should only contain what's there in the text.


A summary of 300 words is an ideal summary.

Summary should be in plain text.


text:  ```{text}```

"""

In [191]:
summary_prompt_template = ChatPromptTemplate.from_template(SUMMARIZE_SYSTEM_CONTEXT)

In [192]:
customer_messages = summary_prompt_template.format_messages(
                    text=qdocs)


In [193]:
customer_messages

[HumanMessage(content="\nYou are an AI designed to summarize text.\n\nYour job is provide a summary of the text text content that is delimited by triple backticks.\nSummary should be pointed to specific points, shouldn't be creative and cover as much as possible the text.\n\nIt Should contains jargons and creative thoughts.\nShould only contain what's there in the text.\n\n\nA summary of 300 words is an ideal summary.\n\nSummary should be in plain text.\n\n\ntext:  ```Microsoft Build: Getting started with generative AI using Azure OpenAI Service\n\nDom Divakaruni, Pablo Castro, Jamie Ovenden, Charlotte Wood, Tim Gordon\n\nMay 2023\n\n\n\n[MUSIC] \n\n\n\nDOM DIVAKARUNI: Well, hello everyone. Great to see you. My name is Dom Divakaruni, I lead Product for Azure OpenAI, and I'm joined by my esteemed colleague, Pablo Castro, who leads Azure Cognitive Search. I'm super happy to be here with you all here to talk about Azure OpenAI and the tools that we're building to help you develop applica

In [194]:
summarized_context = chat(customer_messages)

In [195]:
summarized_context.content

"Microsoft Build: Getting started with generative AI using Azure OpenAI Service\n\nDom Divakaruni, Pablo Castro, Jamie Ovenden, Charlotte Wood, Tim Gordon discuss Azure OpenAI, a platform that brings OpenAI's research to life with enterprise security and privacy. Azure OpenAI runs on Azure Machine Learning, a massive-scale AI platform. The latest GPT-4 model from OpenAI achieves human-level text generation and allows for nuanced instructions.\n\nAzure OpenAI has been used by thousands of customers, including Coursera, which builds AI-powered learning experiences on the platform. Three emerging use cases include AI embedded in products like search, ChatGPT helping novices learn unfamiliar topics, and AI assisting experts in being more efficient.\n\nAnnouncements include four new capabilities: extending the capability of models, incorporating user data, providing more control over service integration, and provision throughput for large-scale production applications. Azure OpenAI Service 

In [196]:
Context = summarized_context.content

In [77]:

TASK_SYSTEM_CONTEXT = """
You are an AI designed to generate datasets needed to train and finetune foundational LLM models.

Your job is to create exhaustive question and answer for the provided text content.
Your question answer pair should cover the context of text.
You should not provide questions and answers specifically about the text as this is part of a document.
Your quetions and answers pairs will be combined to create a dataset for a large document.


You should ouput questions and answers pair in json format. Ensure the response can be parsed by Python json.loads:




You should generate questions and answers solely to cover text content that is delimited by triple backticks.
Your generated questions and answer pairs should be suffient to train another LLM about this text content.

Questions should cover every bit of text. Questions can be many.

text:  ```Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".```

"""



# Make sure the questions are focused on the question types indicated below.
# QUESTION TYPE: {questions_types}
# You can also use below additional context to imporve your questions and answers.
# additional context: {context}


In [78]:
prompt_template = ChatPromptTemplate.from_template(TASK_SYSTEM_CONTEXT)

In [79]:
customer_messages = prompt_template.format_messages(
                    text=docs[1].page_content)


In [80]:
customer_messages

[HumanMessage(content='\nYou are an AI designed to generate datasets needed to train and finetune foundational LLM models.\n\nYour job is to create exhaustive question and answer for the provided text content.\nYour question answer pair should cover the context of text.\nYou should not provide questions and answers specifically about the text as this is part of a document.\nYour quetions and answers pairs will be combined to create a dataset for a large document.\n\n\nYou should ouput questions and answers pair in json format. Ensure the response can be parsed by Python json.loads:\n\n\n\n\nYou should generate questions and answers solely to cover text content that is delimited by triple backticks.\nYour generated questions and answer pairs should be suffient to train another LLM about this text content.\n\nQuestions should cover every bit of text. Questions can be many.\n\ntext:  ```Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singe

In [81]:
customer_response = chat(customer_messages)

In [82]:
print(customer_response.content)

{
  "questions_answers": [
    {
      "question": "What is Beyoncé's full name?",
      "answer": "Beyoncé Giselle Knowles-Carter"
    },
    {
      "question": "When was Beyoncé born?",
      "answer": "September 4, 1981"
    },
    {
      "question": "What is Beyoncé's profession?",
      "answer": "American singer, songwriter, record producer and actress"
    },
    {
      "question": "Where was Beyoncé born and raised?",
      "answer": "Houston, Texas"
    },
    {
      "question": "What did Beyoncé do as a child?",
      "answer": "She performed in various singing and dancing competitions"
    },
    {
      "question": "When did Beyoncé rise to fame?",
      "answer": "In the late 1990s"
    },
    {
      "question": "What was the name of the R&B girl-group Beyoncé was the lead singer of?",
      "answer": "Destiny's Child"
    },
    {
      "question": "Who managed Destiny's Child?",
      "answer": "Mathew Knowles, Beyoncé's father"
    },
    {
      "question": "What 